In [143]:
CUDA_VISIBLE_DEVICES=""

import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import random
import numpy as np
import pandas as pd      
import os
from PIL import Image
import glob
import datetime

%matplotlib inline



import warnings
warnings.filterwarnings('ignore')


print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.11.0


In [152]:
SEED = 42

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

tf.keras.utils.set_random_seed(42)

os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [153]:
img_height = 224
img_width = 224
batch_size = 32
data_dir = "scaled_chest_xray"
train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
    

In [154]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  directory=train_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)


val_ds = tf.keras.utils.image_dataset_from_directory(
  directory=train_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

print(train_ds.class_names)

Found 5232 files belonging to 2 classes.
Using 4186 files for training.
Found 5232 files belonging to 2 classes.
Using 1046 files for validation.
['NORMAL', 'PNEUMONIA']


In [155]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")


In [148]:
def get_model():
  model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(img_width, img_height, 3)),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
  ])
  return model

In [149]:
keras.backend.clear_session()
model = get_model()
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=3e-5), metrics='binary_accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150528)            0         
                                                                 
 dense (Dense)               (None, 100)               15052900  
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 15,053,001
Trainable params: 15,053,001
Non-trainable params: 0
_________________________________________________________________


In [157]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


history = model.fit(train_ds,
          batch_size = batch_size, epochs = 10,
          validation_data=val_ds,
          steps_per_epoch=len(train_ds),
          validation_steps=len(val_ds),
          callbacks=[tensorboard_callback])

Epoch 1/10
131/131 [==============================] - 9s 71ms/step - loss: 4.0546 - binary_accuracy: 0.7341 - val_loss: 3.4406 - val_binary_accuracy: 0.7744
Epoch 2/10
131/131 [==============================] - 11s 80ms/step - loss: 4.0546 - binary_accuracy: 0.7341 - val_loss: 3.4406 - val_binary_accuracy: 0.7744
Epoch 3/10
131/131 [==============================] - 12s 94ms/step - loss: 4.0546 - binary_accuracy: 0.7341 - val_loss: 3.4406 - val_binary_accuracy: 0.7744
Epoch 4/10
131/131 [==============================] - 11s 82ms/step - loss: 4.0546 - binary_accuracy: 0.7341 - val_loss: 3.4406 - val_binary_accuracy: 0.7744
Epoch 5/10
131/131 [==============================] - 11s 85ms/step - loss: 4.0546 - binary_accuracy: 0.7341 - val_loss: 3.4406 - val_binary_accuracy: 0.7744
Epoch 6/10
131/131 [==============================] - 10s 77ms/step - loss: 4.0546 - binary_accuracy: 0.7341 - val_loss: 3.4406 - val_binary_accuracy: 0.7744
Epoch 7/10
131/131 [==============================] -